## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,2020-10-24 17:17:08,42.10,-79.24,44.01,70,90,3.36,overcast clouds
1,1,Bredasdorp,ZA,2020-10-24 17:17:08,-34.53,20.04,66.20,72,39,9.17,scattered clouds
2,2,Rosetta,EG,2020-10-24 17:17:08,31.40,30.42,73.40,60,20,4.70,few clouds
3,3,North Bend,US,2020-10-24 17:17:08,43.41,-124.22,54.00,100,90,3.36,overcast clouds
4,4,Busselton,AU,2020-10-24 17:17:08,-33.65,115.33,59.00,49,69,12.12,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
23,23,Etchoropo,MX,2020-10-24 17:17:11,26.77,-109.67,88.00,47,1,1.99,clear sky
29,29,Aswan,EG,2020-10-24 17:13:24,24.09,32.91,83.98,28,0,18.05,clear sky
39,39,Santiago,CL,2020-10-24 17:14:28,-33.46,-70.65,78.01,40,0,5.82,clear sky
40,40,Viedma,AR,2020-10-24 17:17:13,-40.81,-63.00,84.00,45,74,7.00,broken clouds
41,41,Hobyo,SO,2020-10-24 17:17:14,5.35,48.53,81.45,76,37,18.81,light rain
55,55,Alyangula,AU,2020-10-24 17:17:15,-13.85,136.42,82.40,78,85,8.05,overcast clouds
60,60,Arlit,NE,2020-10-24 17:17:16,18.74,7.39,89.65,11,0,16.64,clear sky
61,61,Hithadhoo,MV,2020-10-24 17:14:55,-0.60,73.08,83.53,75,26,10.54,scattered clouds
67,67,Piranhas,BR,2020-10-24 17:17:17,-16.43,-51.82,86.68,57,99,11.59,light rain
74,74,Aripuana,BR,2020-10-24 17:17:18,-9.17,-60.63,87.13,60,75,7.31,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                204
Date                   205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
all_preferred_cities_df = preferred_cities_df.dropna()
all_preferred_cities_df.count()

City_ID                204
City                   204
Country                204
Date                   204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = all_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
23,Etchoropo,MX,88.00,clear sky,26.77,-109.67,
29,Aswan,EG,83.98,clear sky,24.09,32.91,
39,Santiago,CL,78.01,clear sky,-33.46,-70.65,
40,Viedma,AR,84.00,broken clouds,-40.81,-63.00,
41,Hobyo,SO,81.45,light rain,5.35,48.53,
55,Alyangula,AU,82.40,overcast clouds,-13.85,136.42,
60,Arlit,NE,89.65,clear sky,18.74,7.39,
61,Hithadhoo,MV,83.53,scattered clouds,-0.60,73.08,
67,Piranhas,BR,86.68,light rain,-16.43,-51.82,
74,Aripuana,BR,87.13,light rain,-9.17,-60.63,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
23,Etchoropo,MX,88.00,clear sky,26.77,-109.67,Hotel Villa Mar
29,Aswan,EG,83.98,clear sky,24.09,32.91,Nuba Nile
39,Santiago,CL,78.01,clear sky,-33.46,-70.65,Hotel Panamericano
40,Viedma,AR,84.00,broken clouds,-40.81,-63.00,Hotel Cristal
41,Hobyo,SO,81.45,light rain,5.35,48.53,Osmani Hotel
55,Alyangula,AU,82.40,overcast clouds,-13.85,136.42,Groote Eylandt Lodge
60,Arlit,NE,89.65,clear sky,18.74,7.39,Hôtel Telwa Bungalow
61,Hithadhoo,MV,83.53,scattered clouds,-0.60,73.08,Scoop Guest House
67,Piranhas,BR,86.68,light rain,-16.43,-51.82,Hotel Brasil
74,Aripuana,BR,87.13,light rain,-9.17,-60.63,


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [21]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = (r"C:\Users\Music Matters\Documents\VandyBootcamp\World_Weather_Analysis\Vacation_Search\WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))